# observing-emus
neatened version of notebook `observing-emus.ipynb'

In [1]:
# stock imports
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import scipy
import os
import random
import pickle

##plotting
import matplotlib.pyplot as plt

# plt.style.use('dark_background')
plt.style.use("Solarize_Light2")
plt.rcParams.update({"axes.edgecolor": "black"})
plt.rcParams.update({"text.color": "black"})
plt.rcParams.update({"axes.labelcolor": "black"})
plt.rcParams.update({"xtick.color": "black"})
plt.rcParams.update({"ytick.color": "black"})
plt.rcParams.update({"font.family": "monospace"})

#script imports
from scripts import prior_funcs, utils

from scripts.pitchfuncs_ultra_pca_v2 import emulator
from scripts.pitchfuncs_ultra_pca_v2 import ultra_ns_vector_surface
from scripts.pitchfuncs_ultra_pca_v2 import ultra_ns_popslice
from scripts.pitchfuncs_ultra_pca_v2 import ultra_ns_popwalk


import logging
logging.getLogger('ultranest').setLevel(logging.WARNING)

os.environ["CUDA_VISIBLE_DEVICES"]="1"

physical_devices = tf.config.list_physical_devices("GPU") 

#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


tf.config.experimental.set_memory_growth(physical_devices[0], True)

gpu0usage = tf.config.experimental.get_memory_info("GPU:0")["current"]

2024-10-10 16:59:37.376191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 16:59:37.386989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 16:59:37.390289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 16:59:37.398734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 16:59:37.958057: W tensorflow/compiler/tf2

In [3]:
pitchfork_name = "nu6-40_elu_nonorm_feh"
pitchfork = emulator(pitchfork_name)

with open("pitchfork/" +pitchfork_name+ ".pkl", 'rb') as fp:
     pitchfork_info = pickle.load(fp)

pitchfork_ranges = pitchfork_info['parameter_ranges']

initial_mass range: [min = 0.8, max = 1.2]
initial_Zinit range: [min = 0.003869061466818601, max = 0.0389797119014747]
initial_Yinit range: [min = 0.24, max = 0.32]
initial_MLT range: [min = 1.7, max = 2.5]
star_age range: [min = 0.029664111540787196, max = 13.999973871651315]


## emus with freq unc
applying observational noise to emus, including perturbing the mode frequencies due to the surface term

In [4]:
for emu_idx in range(25):
    path = f"nest/emu{emu_idx}"
    
    hare_df = pd.read_json(path+f"/emu{emu_idx}.json")

    inps = ["initial_mass", "initial_Zinit", "initial_Yinit", "initial_MLT", "star_age"]
    outs = ['calc_effective_T', 'luminosity', 'star_feh'] + [f'nu_0_{i}' for i in range(6,41)]
    
    hare_inps = hare_df[inps]
    
    emu_outs = pitchfork.predict(hare_inps)

    emu_df = hare_df.copy()

    emu_df[outs] = emu_outs

    emu_df.to_json(path+f"/emu{emu_idx}.json")

FileNotFoundError: File nest/emu0/emu0.json does not exist

In [30]:
def nu_max_range(nu_max_n, mode_min=8, mode_max=16):
    modes = np.random.randint(mode_min, mode_max)
    flip = np.random.randint(2)
    int_half = int(modes * 0.5)
    if flip:
        n_min = nu_max_n - int_half
        n_max = nu_max_n + (modes - int_half)
    else:
        n_min = nu_max_n - (modes - int_half)
        n_max = nu_max_n + int_half

    return n_min, n_max


def obs_noise(true, unc, seed=None):
    seeded_random_state = np.random.RandomState(seed=seed)
    rvs_random_states = seeded_random_state.randint(0, high=2**32 - 1, size=len(true))
    noisy_obs = np.empty(len(true))
    idx = 0
    for ob in true:
        noisy_obs[idx] = scipy.stats.norm(loc=ob, scale=unc[idx]).rvs(
            random_state=rvs_random_states[idx]
        )
        idx += 1

    return noisy_obs


def surf_corr(freqs, nu_max, a, b):
    return freqs + a * ((freqs / nu_max) ** b)


inputs = ["initial_mass", "initial_Zinit", "initial_Yinit", "initial_MLT", "star_age", "a", "b"]

teff_unc = 70  # K
luminosity_unc = 0.04  # L\odot
surface_feh_unc = 0.1  # dex

for obs_idx in range(5):
    for emu_idx in range(100):
        path = f"nest/emu{emu_idx}"
        
        emu_df = pd.read_json(path+f"/emu{emu_idx}.json")
    
        nu_max = emu_df["nu_max"].values[0]
        nu_max_n = emu_df["nu_max_n"].values[0]
        n_min, n_max = nu_max_range(nu_max_n)
        outputs = ["calc_effective_T", "luminosity", "star_feh"] + [
            f"nu_0_{i}" for i in range(n_min, n_max + 1)
        ]
    
        emu_df = emu_df[inputs + outputs]
    
        ### add surface correction
        # generate a and b
        a = emu_df["a"].values[0]
        b = emu_df["b"].values[0]
    
        freqs = emu_df[[f"nu_0_{i}" for i in range(n_min, n_max + 1)]].values[0]
    
        dnu = np.mean(freqs[1:] - freqs[:-1])
    
        #nu_max = freqs.mean()
        # shift frequencies
        freqs_corr = surf_corr(freqs, nu_max, a, b)
    
        # reapply
    
        emu_cut = emu_df.copy()
        emu_cut.loc[:, [f"nu_0_{i}" for i in range(n_min, n_max + 1)]] = freqs_corr
    
        frequency_unc = np.random.uniform(0.1, 1)  # \muHz
    
        obs_unc = np.array(
            [teff_unc, luminosity_unc, surface_feh_unc]
            + [frequency_unc + abs(i - nu_max_n) * 0.1 for i in range(n_min, n_max + 1)]
        )
    
        emu_obs = obs_noise(emu_cut.drop(inputs, axis=1).values[0], obs_unc)
    
        emu_obs = obs_noise(emu_cut[outputs].values[0], obs_unc)
        emu_obs_df = emu_cut.copy()
        emu_obs_df[outputs] = emu_obs
        emu_obs_df[["a", "b"]] = [a, b]
    
        # plt.scatter(emu_obs_df[[f'nu_0_{i}' for i in range(n_min, n_max+1)]]%dnu, emu_obs_df[[f'nu_0_{i}' for i in range(n_min, n_max+1)]], label=f'a={a:.2f}, b={b:.2f}, obs unc')
    
        plt.xlim((0, dnu))
        # plt.legend()
        fig, ax = plt.subplots()
        ax.scatter(
            np.arange(0, len(obs_unc)), (emu_obs - emu_cut[outputs].values[0]) / obs_unc
        )
        ax.axhline(0, c="black")
        ax.axhline(-1, c="black", linestyle="--")
        ax.axhline(1, c="black", linestyle="--")
    
        yabs_max = abs(max(ax.get_ylim(), key=abs))
        ax.set_ylim(ymin=-yabs_max, ymax=yabs_max)
        ax.set_xticks(np.arange(0, len(obs_unc)))
        ax.set_xticklabels(outputs)
        # ax.tick_params(axis='x', labelrotation=90)
    
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        ax.set_title("z-score of observed vs true emu params")
        ax.set_ylabel("z-score")
    
        path += f"/obs{obs_idx}"
        
        if not os.path.exists(path):
            os.mkdir(path)
            print(f"{path} created!")
        else:
            print(f"{path} already exists", end="\r")
        emu_obs_df.to_json(path + f"/obs{obs_idx}.json")
        pd.DataFrame([obs_unc], columns=outputs).to_json(path + "/uncs.json")
        plt.savefig(path + "/zscore_plot.png", bbox_inches="tight")
        plt.close()
plt.close()

nest/emu0/obs0 created!
nest/emu1/obs0 created!
nest/emu2/obs0 created!
nest/emu3/obs0 created!
nest/emu4/obs0 created!
nest/emu5/obs0 created!
nest/emu6/obs0 created!
nest/emu7/obs0 created!
nest/emu8/obs0 created!
nest/emu9/obs0 created!
nest/emu10/obs0 created!
nest/emu11/obs0 created!
nest/emu12/obs0 created!
nest/emu13/obs0 created!
nest/emu14/obs0 created!
nest/emu15/obs0 created!
nest/emu16/obs0 created!
nest/emu17/obs0 created!
nest/emu18/obs0 created!
nest/emu19/obs0 created!
nest/emu20/obs0 created!
nest/emu21/obs0 created!
nest/emu22/obs0 created!
nest/emu23/obs0 created!
nest/emu24/obs0 created!
nest/emu25/obs0 created!
nest/emu26/obs0 created!
nest/emu27/obs0 created!
nest/emu28/obs0 created!
nest/emu29/obs0 created!
nest/emu30/obs0 created!
nest/emu31/obs0 created!
nest/emu32/obs0 created!
nest/emu33/obs0 created!
nest/emu34/obs0 created!
nest/emu35/obs0 created!
nest/emu36/obs0 created!
nest/emu37/obs0 created!
nest/emu38/obs0 created!
nest/emu39/obs0 created!
nest/emu40